# First learning attempts
### using Beethoven dataset
* 29 pieces + transpositions across 2 octaves
* ~70h of music (2.7h per transposition)
* 60MB of sparse matrix data -> ~250MB dense

In [23]:
# loading data files names
import os

path = '.\\datasets\\beethoven'
file_names = os.listdir(path)
file_names = list(filter(lambda fn: '.npz' in fn or '.npy' in fn or '.csv' in fn, file_names))
assert len(file_names) > 0, 'Data not found'

f'Found {len(file_names)} files'

'Found 29 files'

In [24]:
# loading data files
from midi_numpy.common import read_numpy_midi
file_paths = [f'{path}\\{fn}' for fn in file_names]
tracks = [read_numpy_midi(fp) for fp in file_paths]

### Creating x and y's

In [33]:
data_x = [t[:-1] for t in tracks]
data_y = [t[1:] for t in tracks]

# allow to free mem
tracks = []

(22355, 22354)

### Processing data

In [35]:
# splitting data into chunks (sequences of equal length)
import numpy as np
CHUNK_LENGTH = 400 # equals to 10s at 0.025s frames

flatten = lambda l: [item for sublist in l for item in sublist]

data_x = flatten([np.array_split(dx, len(dx) // CHUNK_LENGTH + 1) for dx in data_x])
data_y = flatten([np.array_split(dy, len(dy) // CHUNK_LENGTH + 1) for dy in data_y])

len(data_x), len(data_y)

(1109, 1109)

In [36]:
# pad smaller chunks to CHUNK_SIZE
def pad_chunk_sequence(chunk, goal_seq):
    chunk_shape = chunk.shape
    if chunk_shape[0] == goal_seq:
        return chunk
    res = np.zeros((goal_seq, *chunk_shape[1:]))
    res[:chunk_shape[0]] = chunk
    return res

data_x = [pad_chunk_sequence(chunk, CHUNK_LENGTH) for chunk in data_x]
data_y = [pad_chunk_sequence(chunk, CHUNK_LENGTH) for chunk in data_y]

# sanity check
for chunk in data_x:
    assert len(chunk) == CHUNK_LENGTH, 'failed to pad'

In [ ]:
# convert list of matrices to highier dim matrices
data_x = np.stack(data_x)
data_y = np.stack(data_y)

In [ ]:
# dataset generator
def data_gen(batch_size):
    # data shape should be [batch_size, sequence_len, input_dim]
    n_samples = len(data_x)
    while True:
        indices = np.random.randint(0, n_samples, batch_size)
        yield data_x[indices], data_y[indices]